In [27]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

%matplotlib inline

from nltk.tokenize import sent_tokenize
import re

In [28]:
cnn_arg = pd.read_excel('cnn_arg_only_clean.xlsx')
cnn_conc = pd.read_excel('cnn_conclusion_only_clean.xlsx')

In [29]:
from ast import literal_eval

cnn_arg.loc[:,'argum'] = cnn_arg.loc[:,'argum'].apply(lambda x: literal_eval(x))
cnn_arg.loc[:,'polarity_argum'] = cnn_arg.loc[:,'polarity_argum'].apply(lambda x: literal_eval(x))
cnn_arg.loc[:,'subjectivity_argum'] = cnn_arg.loc[:,'subjectivity_argum'].apply(lambda x: literal_eval(x))
cnn_arg.loc[:,'argum_pos'] = cnn_arg.loc[:,'argum_pos'].apply(lambda x: literal_eval(x))
cnn_arg.loc[:,'argum_neg'] = cnn_arg.loc[:,'argum_neg'].apply(lambda x: literal_eval(x))
cnn_arg.loc[:,'argum_neu'] = cnn_arg.loc[:,'argum_neu'].apply(lambda x: literal_eval(x))
cnn_arg.loc[:,'argum_compound'] = cnn_arg.loc[:,'argum_compound'].apply(lambda x: literal_eval(x))

cnn_conc.loc[:,'conclusion'] = cnn_conc.loc[:,'conclusion'].apply(lambda x: literal_eval(x))
cnn_conc.loc[:,'polarity_conclusion'] = cnn_conc.loc[:,'polarity_conclusion'].apply(lambda x: literal_eval(x))
cnn_conc.loc[:,'subjectivity_conclusion'] = cnn_conc.loc[:,'subjectivity_conclusion'].apply(lambda x: literal_eval(x))
cnn_conc.loc[:,'conclusion_pos'] = cnn_conc.loc[:,'conclusion_pos'].apply(lambda x: literal_eval(x))
cnn_conc.loc[:,'conclusion_neg'] = cnn_conc.loc[:,'conclusion_neg'].apply(lambda x: literal_eval(x))
cnn_conc.loc[:,'conclusion_neu'] = cnn_conc.loc[:,'conclusion_neu'].apply(lambda x: literal_eval(x))
cnn_conc.loc[:,'conclusion_compound'] = cnn_conc.loc[:,'conclusion_compound'].apply(lambda x: literal_eval(x))

In [55]:
cnn_arg.head(3)

,argum_pos,argum_neg,argum_neu,argum_compound,polarity_argum,subjectivity_argum,text_id,argum
0,[0.077],[0.215],[0.707],[-0.5106],[0.5],[0.7],4,"[he said he is ""innocent of all charges and th..."
1,[0.0],[0.0],[1.0],[0.0],[0.0],[0.0],7,"[so in 1997, gebre returned to ethiopia with a..."
2,[0.0],[0.045],[0.955],[-0.2411],[0.5],[0.5],9,['a full-blown dictatorship' dss officials sa...


In [59]:
cnn_arg['argum'].iloc[5]

['after the story broke in the media, osagie faced a university panel investigating the allegations on the tape and is awaiting its findings so she can receive her certificate.',
 'student speaks out  in her only interview since the audio was leaked online, osagie, 23, told cnn she had developed a mentor-mentee relationship with the professor after she helped him edit his book.',
 "her lawyer, abiola akiyode-afolabi, told cnn more victims have contacted her with allegations about the same professor since hearing about osagie's case.",
 '— remi sonaiya (@oluremisonaiya) april 9, 2018  the practice thrives because students do not come forward with these allegations and also because they do not trust their institutions to handle such matters with discretion, she added.',
 'but the relationship soon made her uncomfortable because he started to make sexual advances towards her, she said.',
 '"this is because of the caliber of the lecturer involved," he added.',
 'so, i decided to record our

In [30]:
filepath = "NRC-emotion-lexicon-wordlevel-alphabetized-v0.92.txt"
emolex_df = pd.read_csv(filepath,  names=["word", "emotion", "association"], skiprows=45, sep='\t')
emolex_df = emolex_df.pivot(index='word', columns='emotion', values='association').reset_index()
emolex_df.head()

emotion,word,anger,anticipation,disgust,fear,joy,negative,positive,sadness,surprise,trust
0,NaN,0,0,0,0,0,0,0,0,0,0
1,aback,0,0,0,0,0,0,0,0,0,0
2,abacus,0,0,0,0,0,0,0,0,0,1
3,abandon,0,0,0,1,0,1,0,1,0,0
4,abandoned,1,0,0,1,0,1,0,1,0,0


In [31]:
# list of fear words
fear_words = emolex_df[emolex_df.fear == 1]['word']

from sklearn.feature_extraction.text import TfidfVectorizer
# tuning the vectorizer parameters
vectorizer = TfidfVectorizer(vocabulary=fear_words,use_idf=False)

In [33]:
def get_fear(sent):
    vectorizer = TfidfVectorizer(vocabulary=fear_words,use_idf=False)
    i= vectorizer.fit_transform(sent)
    x= i.toarray()
    return(x.sum(axis=1))

In [34]:
cnn_arg['fear'] = cnn_arg.argum.apply(get_fear)
cnn_conc['fear'] = cnn_conc.conclusion.apply(get_fear)

In [36]:
def get_total_fear(x):
    for i in x:
        i.sum()

In [37]:
cnn_arg['total_fear'] = cnn_arg.fear.apply(sum)
cnn_conc['total_fear'] = cnn_conc.fear.apply(sum)

In [38]:
fearless= pd.DataFrame()

fearless[['appeal_to_fear','fear_indicator','text_id']] = cnn_arg[['argum','total_fear','text_id']]

cnn_arg.drop(columns=['fear','total_fear'],inplace=True)

# creating a limit / statistical approach
differ = (fearless.fear_indicator.mean() + (fearless.fear_indicator.std()))*2

fear_index = fearless[fearless['fear_indicator'].apply(lambda x: x < differ)].index

fear_fallacy= fearless.drop(fear_index)

In [43]:
# check what we got
fear_fallacy

,appeal_to_fear,fear_indicator,text_id
23,[those five demands are: withdraw the extradit...,10.934636,37
25,[the officer fired the gun because he believed...,12.715799,41
26,[but scientists have since found dna evidence ...,8.511308,42
52,"[their aim is to make visible the victims of ""...",9.974691,85
82,[they give them a bag of food and people sell ...,9.009968,125
83,[when cnn approached a number of reputable aid...,9.285472,126
123,[here's why it's legal birth of a cult brand ...,8.242641,183
124,"[cartoonists, get arrested, put in jail everyw...",10.242641,184
153,"[below the photo is written: ""because one holo...",12.242641,229
154,"[over the years, some who had these surgeries ...",9.828427,230


In [44]:
# qualititive inspection
fear_fallacy['appeal_to_fear'].iloc[2]

["but scientists have since found dna evidence that the plague could have existed much further back than previously thought -- there's evidence it existed in europe some 5,000 years ago  and the idea that the second pandemic, the black death, could have started in china is unlikely, black said.",
 "so mao put in place a number of measures to control the country's rampant disease.",
 "but that only led to another question: if the disease wasn't genetically different, then why was the second pandemic so deadly?",
 '"it\'s so central to western identity," he said.',
 "yunnan was hit by another breakout between 1986 and 2005 , and another case was diagnosed in yunnan in 2016.  why we're so fascinated by the plague  centuries on from the black death, people around the world continue to be transfixed by the plague in a way they're not by other diseases.",
 'chinese cultural revolution poster about the so-called four pests: mosquitoes, rats, flies and sparrows.',
 '"it\'s part of our past, wh

In [ ]:
# # saving the output
# fear_fallacy.to_excel('fear/appeal_to_fear_fallacy.xlsx',index= False)